In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
import sqlite3


In [ ]:
data = pd.read_html("foggia_2024.html", encoding="utf-8", decimal=".")
data = data[0]

In [ ]:
def load_db_sql(table_name: str = 'stats', db_name="fm_estatistica.db") -> pd.DataFrame:
    
    conn = sqlite3.connect('fm_estatistica.db')
    
    
    try:
        df = pd.read_sql(f"SELECT * FROM {table_name}", conn)
        # debug_print(Fore.GREEN + f"[INFO]: df carregada com sucesso." + Style.RESET_ALL)
        
        try:
            df['posicao_analise'] = df['posicao_analise'].apply(lambda x: x.split("."))
            # debug_print(Fore.GREEN + f"[INFO]: Coluna 'posicao_analise' convertida com sucesso." + Style.RESET_ALL)
            
        except Exception as e:
            pass
            # debug_print(Fore.RED + f"[ERROR]: Erro ao converter coluna 'posicao_analise': {str(e)}" + Style.RESET_ALL)
        
        return df
    
    except Exception as e:
        # debug_print(Fore.RED + f"[ERROR]: Erro ao carregar df: {str(e)}" + Style.RESET_ALL)
        
        return pd.DataFrame()

In [ ]:
df_proc = load_db_sql()

In [ ]:
df_proc.to_excel(excel_writer="data.xlsx",index=False)

In [ ]:
df_goleadores = df_proc[df_proc['gols'] >= 15]

In [ ]:
colunas = [
    'nome',
    'clube_atual',
    'gols_esperados_sem_penaltis_p90',
    'gols_sem_penaltis_p90',
    'gols_esperados_sem_penalti_por_chutes_sem_penalti',
    'chutes_sem_penaltis_p90',
    'chutes_no_gol_sem_penalti_p90',
    'perigo_das_finalizacoes'
    ]

colunas_rename = {
    'gols_sem_penaltis_p90':'npg',
    'gols_esperados_sem_penaltis_p90':'xnpg',
    'gols_esperados_sem_penalti_por_chutes_sem_penalti':'xnpg_per_shot',
    'chutes_sem_penaltis_p90':'np_sht_p90',
    'chutes_no_gol_sem_penalti_p90':'np_shtG_p90',
    }

In [ ]:
df_goleadores[colunas].sort_values(by='gols_sem_penaltis_p90', ascending=False).rename(columns=colunas_rename)

In [ ]:
import re
def _get_season(path:str) -> int:
    
    pattern = r'^(?P<nome>.+_*)(?P<season>[0-9]{4})(?P<ext>.{1}[a-zA-Z0-9]+)'
    season = re.search(pattern, path).group('season')
    
    return int(season)

In [ ]:

path = 'Italia_254024.html'

_get_season(path)

In [ ]:
import pandas as pd

data = pd.read_html('italia_2024.html', encoding="utf-8", decimal=".")
df = pd.DataFrame(data[0])

In [ ]:
df

In [ ]:
from data_manipulation import*
from data_preprocessing import*
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

_df = fm_create_dataframe(path='teste_2024.html')

In [ ]:
_df['np_chutes_gol_p90'].where(_df['divisao'].str.contains('Serie C NOW Girone',case=False,na=False)).describe()

In [ ]:
_df[['nome','idade','posicao_analise','divisao','aval_fin','npxG_p90','npG_p90','npxG_per_np_chute']].where(_df['divisao'].str.contains('Serie C NOW Girone',case=False,na=False)).sort_values(by=['aval_fin'],ascending=False)

In [ ]:
from fm_analycts_parametrs import DEF_STATS
ordem = ['nome', 'clube','divisao','posicao_analise','idade','minutos'] + DEF_STATS
_df = _df[ordem].sort_values(by=['aval_def'],ascending=False)

In [ ]:
_df['divisao'].unique().tolist()

In [ ]:
div_med = _df.groupby('divisao')['aval_def'].mean()
div_desv = _df.groupby('divisao')['aval_def'].std()

In [ ]:
_df_zagueiros = _df.copy()
_df_zagueiros = _df_zagueiros[_df['posicao_analise'].apply(lambda x: 'Zagueiro' in x)]

In [ ]:
_df_zagueiros = _df.copy()
_df_zagueiros = _df_zagueiros[_df['posicao_analise'].apply(lambda x: 'Zagueiro' in x)]

_df_zagueiros['aval_def_adj'] = _df_zagueiros.groupby('divisao')['aval_def'].transform(lambda x: (x - x.mean()) / x.std())
_df_zagueiros['rtg_adef_adj'] = _df_zagueiros.groupby('divisao')['rtg_adef'].transform(lambda x: (x - x.mean()) / x.std())
_df_zagueiros['rtg_des_adj'] = _df_zagueiros.groupby('divisao')['rtg_des'].transform(lambda x: (x - x.mean()) / x.std())
_df_zagueiros['rtg_duel_adj'] = _df_zagueiros.groupby('divisao')['rtg_duel'].transform(lambda x: (x - x.mean()) / x.std())
_df_zagueiros['rtg_jg_ar_adj'] = _df_zagueiros.groupby('divisao')['rtg_jg_ar'].transform(lambda x: (x - x.mean()) / x.std())

_df_zagueiros['aval_def_adj1'] = np.nan
    
p1 = 0.20
p2 = 0.15
p3 = 0.15
p4 = 0.25
p5 = 0.25
    
_df_zagueiros['aval_def_adj1'] = (((     (_df_zagueiros['rtg_jg_ar_adj']*_df_zagueiros['jg_ar_t_p90']*p1) +
                                        _df_zagueiros['rtg_duel_adj']*p2 + 
                                        (_df_zagueiros['rtg_des_adj']*_df_zagueiros['des_t_p90'])*p3 + 
                                        (_df_zagueiros['poss_g_p90']/_df_zagueiros['adef_t_p90'])*p4+
                                        _df_zagueiros['rtg_adef_adj']*p5
                                        ))).round(2)
    
_df_zagueiros['aval_def_adj1'] = _df_zagueiros['aval_def_adj1'].replace([np.inf,-np.inf ], np.nan)
_df_zagueiros['aval_def_adj2'] = (_df_zagueiros['aval_def_adj1'] / 4).replace([np.inf,-np.inf ], np.nan)

print(_df_zagueiros.groupby('divisao')['rtg_adef'].mean())
print(_df_zagueiros.groupby('divisao')['rtg_adef'].std())

_df_zagueiros[['nome','clube','divisao','posicao_analise','aval_def', 'aval_def_adj']].sort_values(by=['aval_def_adj'],ascending=False)

_df_zagueiros[(_df_zagueiros['divisao'].str.contains('Serie C NOW')) & (_df_zagueiros['minutos'] >= 900) & (_df_zagueiros['posicao_analise'].apply(lambda x: 'Zagueiro' in x))][[
                                        'nome',
                                        'clube',
                                        'divisao',
                                        'minutos',
                                        'posicao_analise',
                                      'aval_def', 
                                      'aval_def_adj',
                                      'aval_def_adj1',
                                      'aval_def_adj2',
                                      'rtg_adef',
                                      'rtg_adef_adj',
                                      'rtg_des',
                                      'rtg_des_adj',
                                      'rtg_duel',
                                      'rtg_duel_adj',
                                      'rtg_jg_ar',
                                      'rtg_jg_ar_adj']].sort_values(by=['aval_def'],ascending=False)

In [ ]:
_df['aval_def_adj'] = _df.groupby('divisao')['aval_def'].transform(lambda x: (x - x.mean()) / x.std())
_df['rtg_adef_adj'] = _df.groupby('divisao')['rtg_adef'].transform(lambda x: (x - x.mean()) / x.std())
_df['rtg_des_adj'] = _df.groupby('divisao')['rtg_des'].transform(lambda x: (x - x.mean()) / x.std())
_df['rtg_duel_adj'] = _df.groupby('divisao')['rtg_duel'].transform(lambda x: (x - x.mean()) / x.std())
_df['rtg_jg_ar_adj'] = _df.groupby('divisao')['rtg_jg_ar'].transform(lambda x: (x - x.mean()) / x.std())

In [ ]:
_df['aval_def_adj1'] = np.nan
    
p1 = 0.20
p2 = 0.15
p3 = 0.15
p4 = 0.25
p5 = 0.25
    
_df['aval_def_adj1'] = (((     (_df['rtg_jg_ar_adj']*_df['jg_ar_t_p90']*p1) +
                                        _df['rtg_duel_adj']*p2 + 
                                        (_df['rtg_des_adj']*_df['des_t_p90'])*p3 + 
                                        (_df['poss_g_p90']/_df['adef_t_p90'])*p4+
                                        _df['rtg_adef_adj']*p5
                                        ))).round(2)
    
_df['aval_def_adj1'] = _df['aval_def_adj1'].replace([np.inf,-np.inf ], np.nan)
_df['aval_def_adj2'] = (_df['aval_def_adj1'] / 4).replace([np.inf,-np.inf ], np.nan)

In [ ]:
_df[['nome','clube','divisao','posicao_analise','aval_def', 'aval_def_adj']].sort_values(by=['aval_def_adj'],ascending=False)

_df[(_df['divisao'] == 'Serie A TIM') & (_df['minutos'] >= 900) & (_df['posicao_analise'].apply(lambda x: 'Zagueiro' in x))][[
                                        'nome',
                                        'clube',
                                        'divisao',
                                        'posicao_analise',
                                      'aval_def', 
                                      'aval_def_adj',
                                      'aval_def_adj1',
                                      'aval_def_adj2',
                                      'rtg_adef',
                                      'rtg_adef_adj',
                                      'rtg_des',
                                      'rtg_des_adj',
                                      'rtg_duel',
                                      'rtg_duel_adj',
                                      'rtg_jg_ar',
                                      'rtg_jg_ar_adj']].sort_values(by=['aval_def_adj2'],ascending=False)

In [ ]:
_df[ordem].sort_values(by=['aval_def'],ascending=False)